In [11]:
import torch
dump = torch.load("rnnt_joint_dump.pt", map_location="cpu", weights_only=True )

tj = dump["teacher_joint"]          # (B,T,U,V)
sj = dump["student_joint"]

# e.g. compute per‑position softmax
T = 20
teacher_p = torch.softmax(tj / T, dim=-1)
student_lp = torch.log_softmax(sj / T, dim=-1)
kl_intermidiate = torch.nn.functional.kl_div(student_lp, teacher_p,
                                reduction="none").sum(-1)

kl = kl_intermidiate.mean(dim=(1, 2)) * (T**2)

offline_kl = kl.mean()   

print("Intermidiate KL:", kl)
print("Offline KL:", offline_kl.item())

Intermidiate KL: tensor([26.2418, 31.6345, 22.5171, 18.8390, 30.3760, 30.2018, 25.5498, 29.3478],
       grad_fn=<MulBackward0>)
Offline KL: 26.83846092224121


In [6]:
kl_pos = torch.nn.functional.kl_div(student_lp, teacher_p, reduction='none')  # (B, T, U, V)
kl_sum = kl_pos.sum(-1)  # (B, T, U)

print(kl_sum)

kl_per_sample = kl_sum.mean(dim=(1, 2))  # (B,)
print("KL per sample:", kl_per_sample)

tensor([[[0.6641, 0.7880, 0.9131,  ..., 0.2978, 0.2841, 0.2924],
         [0.7909, 0.8251, 0.9037,  ..., 0.3590, 0.3458, 0.3526],
         [0.9411, 0.8119, 0.8889,  ..., 0.3997, 0.3854, 0.3902],
         ...,
         [0.4873, 0.7092, 0.7697,  ..., 0.2168, 0.2098, 0.2115],
         [0.4885, 0.7065, 0.7717,  ..., 0.2171, 0.2104, 0.2105],
         [0.4818, 0.7115, 0.7699,  ..., 0.2136, 0.2061, 0.2083]],

        [[0.7898, 0.5736, 0.4656,  ..., 0.4037, 1.3118, 0.4570],
         [1.2515, 0.8188, 0.5717,  ..., 0.4686, 1.4619, 0.7413],
         [2.1305, 2.3063, 1.8055,  ..., 0.9677, 1.7195, 1.8705],
         ...,
         [0.4887, 0.6136, 0.4712,  ..., 0.3478, 1.1583, 0.3914],
         [0.4889, 0.6123, 0.4704,  ..., 0.3464, 1.1613, 0.3917],
         [0.4823, 0.6078, 0.4667,  ..., 0.3444, 1.1644, 0.3850]],

        [[0.3498, 0.2992, 0.2607,  ..., 0.5040, 0.8839, 0.2336],
         [0.3362, 0.2921, 0.2907,  ..., 0.5095, 0.9556, 0.2789],
         [0.3356, 0.3014, 0.3032,  ..., 0.5148, 0.9709, 0.